In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
# import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import gutenberg, stopwords
from collections import Counter

In [2]:
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)

In [3]:
sent_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
alice_sentences = sent_tokenizer.tokenize(alice)
persuasion_sentences = sent_tokenizer.tokenize(persuasion)

sentences = pd.DataFrame(alice_sentences + persuasion_sentences)
sentences.head()

,0
0,Alice was beginning to get very tired of sitti...
1,So she was considering in her own mind (as wel...
2,There was nothing so VERY remarkable in that; ...
3,Oh dear!
4,I shall be late!'


In [4]:
alice_sents = [[sent, "Carroll"] for sent in alice_sentences]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_sentences]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,Alice was beginning to get very tired of sitti...,Carroll
1,So she was considering in her own mind (as wel...,Carroll
2,There was nothing so VERY remarkable in that; ...,Carroll
3,Oh dear!,Carroll
4,I shall be late!',Carroll


In [5]:
from nltk.tokenize import word_tokenize
from collections import Counter
from nltk.stem.wordnet import WordNetLemmatizer

lmtzr= WordNetLemmatizer()

import string 


def bag_of_words(text, include_stop=False):
    tokenize = word_tokenize(text)
    # Build a list of words.
    # Strip out punctuation and, optionally, stop words.
    words = []
    my_stopwords = stopwords.words('english')
    for token in tokenize:
        if (token not in my_stopwords or include_stop) and (token not in string.punctuation):
            words.append(lmtzr.lemmatize(token))
            
    # Build and return a Counter object containing word counts.
    return [item[0] for item in Counter(words).most_common(2000)]

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words, include_stop = False):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    my_stopwords = stopwords.words('english')
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        tokenize = word_tokenize(sentence)
        words = []
        
        for token in tokenize:
            if (token not in my_stopwords or include_stop) and (token not in string.punctuation)and lmtzr.lemmatize(token) in common_words:
                words.append(lmtzr.lemmatize(token))   
            # Populate the row with word counts.
            for word in words: 
                df.loc[i, word] += 1
                #df.loc[i, :] = 

        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
    
    return df

# Set up the bags.
alicewords = bag_of_words(alice)
persuasionwords = bag_of_words(persuasion)

common_words = set(alicewords + persuasionwords)

In [6]:
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500
Processing row 5000


,seek,wait,beyond,pleasing,'Repeat,walked,autumn,nay,love,shoe,...,ventured,lit,lazily,Nobody,sofa,playing,roof,'That,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Alice was beginning to get very tired of sitti...,Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,So she was considering in her own mind (as wel...,Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,There was nothing so VERY remarkable in that; ...,Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Oh dear!,Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,I shall be late!',Carroll


In [7]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier()
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 0.993057746923

Test set score: 0.912446758164


In [8]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(3169, 3197) (3169,)
Training set score: 0.994004417797

Test set score: 0.913866540464


In [9]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.893972862102

Test set score: 0.867486985329


In [10]:
emma = gutenberg.raw('austen-emma.txt')
emma = re.sub(r'VOLUME \w+', '', emma)
emma = re.sub(r'CHAPTER \w+', '', emma)
emma = text_cleaner(emma)
print(emma[:100])

Emma Woodhouse, handsome, clever, and rich, with a comfortable home and happy disposition, seemed to


In [11]:
sent_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
emma_sentences = sent_tokenizer.tokenize(emma)
persuasion_sentences = sent_tokenizer.tokenize(persuasion)

# Group into sentences.
emma_sents = [[sent, "Austen"] for sent in emma_sentences]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_sentences]

emma_sents = emma_sents[0:len(alice_sents)]

In [12]:
emma_sentences = pd.DataFrame(emma_sents)
emma_bow = bow_features(emma_sentences, common_words)

print('done')

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
done


In [13]:
X_Emma_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    emma_bow.drop(['text_sentence','text_source'], 1)
), axis=0)
y_Emma_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Austen'] * emma_bow.shape[0])])

# Model.
print('\nTest set score:', lr.score(X_Emma_test, y_Emma_test))
lr_Emma_predicted = lr.predict(X_Emma_test)
pd.crosstab(y_Emma_test, lr_Emma_predicted)


Test set score: 0.687883435583


col_0,Austen,Carroll
row_0,,
Austen,1493,132
Carroll,682,301


In [14]:
from sklearn import svm
svc = svm.SVC()

train = svc.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', svc.score(X_train, y_train))
print('\nTest set score:', svc.score(X_test, y_test))

(3169, 3197) (3169,)
Training set score: 0.878826128116

Test set score: 0.804070042593


In [15]:
for i, sentence in enumerate(word_counts['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
    list_of_tokens = word_tokenize(sentence)
    word_counts.loc[i,'sentence_length'] = len(list_of_tokens)

In [16]:
word_counts["sentence_length"] = word_counts['text_sentence'].apply(lambda x: len(word_tokenize(x)))
word_counts['punc_count'] = word_counts['text_sentence'].apply(lambda x: sum([token in string.punctuation for token in word_tokenize(x)]))

In [17]:
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)


lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(3169, 3199) (3169,)
Training set score: 0.99526664563

Test set score: 0.921438712731


In [18]:
gutenberg.fileids()

['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt',
 'shakespeare-hamlet.txt',
 'shakespeare-macbeth.txt',
 'whitman-leaves.txt']

In [19]:
macbeth = gutenberg.raw('shakespeare-macbeth.txt')
macbeth = re.sub(r'VOLUME \w+', '', macbeth)
macbeth = re.sub(r'CHAPTER \w+', '', macbeth)
macbeth = text_cleaner(macbeth)
print(macbeth[:100])

Actus Primus. Scoena Prima. Thunder and Lightning. Enter three Witches. 1. When shall we three meet 


In [20]:
sent_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
# emma_sentences = sent_tokenizer.tokenize(emma)
# persuasion_sentences = sent_tokenizer.tokenize(persuasion)
alice_sentences = sent_tokenizer.tokenize(alice)
macbeth_sentences = sent_tokenizer.tokenize(macbeth)

# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_sentences]
macbeth_sents = [[sent, "Shakespeare"] for sent in macbeth_sentences]

In [21]:
macbeth_sentences = pd.DataFrame(macbeth_sents)
macbeth_bow = bow_features(macbeth_sentences, common_words)
print('done')

Processing row 0
Processing row 500
Processing row 1000
done


In [22]:
macbeth_bow["sentence_length"] = macbeth_bow['text_sentence'].apply(lambda x: len(word_tokenize(x)))
macbeth_bow['punc_count'] = macbeth_bow['text_sentence'].apply(lambda x: sum([token in string.punctuation for token in word_tokenize(x)]))

In [23]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split


Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)

In [24]:
X_Macbeth_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    macbeth_bow.drop(['text_sentence','text_source'], 1)
), axis=0)
y_Macbeth_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Shakespeare'] * macbeth_bow.shape[0])])

# Model.
print('\nTest set score:', lr.score(X_Macbeth_test, y_Macbeth_test))
lr_Macbeth_predicted = lr.predict(X_Macbeth_test)
pd.crosstab(y_Macbeth_test, lr_Macbeth_predicted)


Test set score: 0.123774509804


col_0,Austen,Carroll
row_0,,
Carroll,680,303
Shakespeare,927,538


In [25]:
Carroll = word_counts[word_counts['text_source'] == 'Carroll']



X = np.concatenate((Carroll.drop(['text_sentence','text_source'], 1),
    macbeth_bow.drop(['text_sentence','text_source'], 1)))
Y = np.concatenate((Carroll['text_source'],
    macbeth_bow['text_source']),axis=0)

X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size=0.4,random_state=0)

lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(1854, 3199) (1854,)
Training set score: 0.988133764833

Test set score: 0.892394822006
